In [1]:
import numpy as np
import pandas as pd
import graph_tool.all as gt
import matplotlib.pyplot as plt
import pickle
import sys

In [2]:
gt.__version__

'2.91 (commit 5f147344, )'

In [3]:
import os

# Specify the directory path
directory_path = 'states/'

# Get all files in the directory
file_names = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]

# print(file_names)


In [4]:
file_names.sort()

In [5]:
file_names

['Topic_ALL__Post90_False__Randomize_Direction_False__Remove_Local_Field_True__Negative_Weights_Allowed_False_371d00e5-6e81-43ca-b53a-2dba56bdd866.pickle']

In [6]:
pickle_file = file_names[0]

In [7]:
pickle_file

'Topic_ALL__Post90_False__Randomize_Direction_False__Remove_Local_Field_True__Negative_Weights_Allowed_False_371d00e5-6e81-43ca-b53a-2dba56bdd866.pickle'

In [8]:
# pickle_file = 'Topic_ALL__Post90_True__Randomize_Direction_False__Remove_Local_Field_True__Negative_Weights_Allowed_False_f69c5ce0-7090-413a-bde9-cbc9234f1e96.pickle'

In [9]:
with open(directory_path + pickle_file, 'rb') as fout:
        state_dict = pickle.load(fout)

In [10]:
def bytes_to_mb(size_in_bytes):
    return size_in_bytes / (1024 ** 2)

In [11]:
# Step 1: Read the CSV file into a DataFrame
iso_df = pd.read_csv('data/alpha-3.csv')

# Step 2: Create a dictionary to map country names to their ISO codes
iso_dict = pd.Series(iso_df['alpha-3'].values, index=iso_df['Country']).to_dict()

In [12]:
topic_labels = {
    'me': 'Israel-Palestine Conflict',
    'nu': 'Nuclear Weapons',
    'di': 'Arms Control',
    'co': 'Colonialism',
    'hr': 'Human Rights',
    'ec': 'Economic Development',
    'ALL': 'All Resolutions'
}

In [13]:
# Load the saved dictionary (assuming you already have `state_dict`)
# state_dict = ...

# Unpack each item in state_dict as a variable with the same name
for key, value in state_dict.items():
    globals()[key] = value  # This will create variables with the same names as the dictionary keys


In [14]:
state_title_filename

'Topic_ALL__Post90_False__Randomize_Direction_False__Remove_Local_Field_True__Negative_Weights_Allowed_False_371d00e5-6e81-43ca-b53a-2dba56bdd866'

In [15]:
if 'country' not in list( g.vp.keys() ):
    print('yes')

    countries = pd.read_csv('countries_list_temp.csv')
    
    countries = list( countries['0'] )
    
    vprop_label = g.new_vertex_property("string")
    
    for i in range( len(countries) ):
        vprop_label[ g.vertex(i) ] = countries[i]
    
    # bstate = state.get_block_state()
    
    g.vp.country = vprop_label

yes


In [16]:
g = gt.GraphView(g, efilt=lambda e: g.ep['eprob'][e] > 0.5)

g.num_edges()

g = gt.GraphView(g, efilt=lambda e: g.ep['x'][e] >g.ep['x'].a.mean())

g.clear_filters()

g.num_edges()

4209

In [17]:
nested_blockstate_g = gt.NestedBlockState( g, bs = state.nbstate.get_bs().copy() )

In [18]:
pmode = gt.PartitionModeState(bs[-4000:], converge=True)
pv = pmode.get_marginal(g)

In [19]:
nested_blockstate_g_pv = nested_blockstate_g.g.new_vertex_property("vector<int32_t>")

In [20]:
for v in nested_blockstate_g.g.vertices():
    nested_blockstate_g_pv[v] = pv[v]
    # print (nested_blockstate_g_pv[v] )

In [21]:
# Step 1: Create a list of tuples with (vertex, max_index, normalized proportion at max index)
node_info = []
max_indices = []
vertex_orders = []
for v in g.vertices():
    values = np.array(nested_blockstate_g_pv[v])
    max_index = np.argmax(values)  # Find the max index
    normalized_values = values / values.sum()  # Normalize the values
    max_proportion = normalized_values[max_index]  # Proportion at the max index
    node_info.append((int(v), max_index, max_proportion))  # Append vertex id and computed values
    max_indices.append( max_index )
    vertex_orders.append( max_proportion )
# Step 2 and Step 3: Sort by max index, then by max proportion in descending order
# sorted_nodes = sorted(node_info, key=lambda x: (x[1], -x[2]))

# Extract just the vertex order and the max indices in the desired order
# vertex_order = [node[0] for node in sorted_nodes]
# max_indices = [node[1] for node in sorted_nodes]  # Store max_index for each vertex

# max_indices = np.array( max_indices )
# Output the ordered vertex indices and max indices
# print("Ordered vertex indices:", vertex_order)
# print("Max indices for each vertex in order:", max_indices)


# temp_state = state.nbstate.get_bs()

# nested_blockstate_g.get_state()

# max_indices

# vertex_block_property = nested_blockstate_g.g.new_vertex_property("int")

# vertex_block_property.a[:] = max_indices
 
# vertex_block_property.a

# temp_state[0] = vertex_block_property

# temp_state

# nested_blockstate_g.set_state( temp_state )

In [22]:
vertex_order_property = nested_blockstate_g.g.new_vertex_property("float")

vertex_order_property.a[:] = vertex_orders

In [23]:
def shuffle_categories(arr):
    """
    Shuffle categories in the array while preserving their structure.
    
    Parameters:
        arr: List or 1D array of integers representing categories.
    
    Returns:
        A new array with categories shuffled.
    """
    # Step 1: Find unique categories
    unique_values = np.unique(arr)
    
    # Step 2: Shuffle the unique categories
    shuffled_values = unique_values.copy()
    np.random.shuffle(shuffled_values)
    
    # Step 3: Create a mapping from original to shuffled categories
    value_map = {original: shuffled for original, shuffled in zip(unique_values, shuffled_values)}
    
    # Step 4: Replace values in the array according to the shuffled mapping
    shuffled_array = np.array([value_map[val] for val in arr])
    
    return shuffled_array

max_indices = shuffle_categories(max_indices)

# country_colors = state.bstate.get_blocks().a
country_colors = max_indices

In [24]:
new_block_state = nested_blockstate_g.get_bs().copy()
new_block_state[0].a[:] = max_indices
nested_blockstate_g.set_state( new_block_state )

In [25]:
# pmode = gt.PartitionModeState(bs, converge=True)
# pv = pmode.get_marginal(g)

for v in nested_blockstate_g.g.vertices():
    nested_blockstate_g_pv[v] = pv[v]


In [26]:
# edge width
ew = nested_blockstate_g.g.ep.eprob.copy()
# ew.a = abs(ew.a - .5)

In [27]:
eorder = ew.copy()
eorder.a = -eorder.a

In [28]:
plt.close("all")           # Close all open figures
# plt.switch_backend("agg")   # Revert to the default backend
plt.switch_backend("module://matplotlib_inline.backend_inline")  # Set to Jupyter inline backend


In [29]:
edge_pen_width = gt.prop_to_size(ew, 0.8, 3, power=1, log = True)

In [30]:
# plt.hist( edge_pen_width.fa, 100 );

In [31]:
import matplotlib

In [32]:
# cnorm = matplotlib.colors.Normalize(vmin=-abs(ew.fa).max(), vmax=abs(ew.fa).max())
# cnorm = matplotlib.colors.Normalize(vmin=abs(ew.fa).min(), vmax=abs(ew.fa).max())
cnorm = matplotlib.colors.Normalize(vmin=0.2, vmax=abs(ew.fa).max())

In [33]:
vertex_block_property = nested_blockstate_g.g.new_vertex_property("int")

vertex_block_property.a[:] = max_indices


In [34]:
## plt.close("all")           # Close all open figures
plt.switch_backend("cairo")   # to enable vector drawing

nested_blockstate_g.draw( #vertex_shape="pie",
                        edge_color = nested_blockstate_g.g.ep.x,
                        vertex_fill_color = vertex_block_property,
                        # ecmap=red_cm,
                        # ecmap=matplotlib.cm.coolwarm_r,
                        ecmap=matplotlib.cm.Blues,
                        ecnorm=cnorm,
                        edge_gradient =[],
                        # vertex_pie_fractions=nested_blockstate_g_pv,
                        rel_order = vertex_order_property,
                        edge_pen_width=edge_pen_width,
                        deg_size = False,
                        vertex_size = 10,
                        hedge_color = 'k',
                        hvertex_fill_color = 'k',
                        eorder = nested_blockstate_g.g.ep.x,
                        vertex_text = vprop_label,
                        output='./' + "stabilization -names-" + state_title_filename +'-.pdf',
                        output_size = (1200, 1200)
                        )

(<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7f7872908620, at 0x7f7863fefc80>,
 <GraphView object, directed, with 206 vertices and 205 edges, edges filtered by <EdgePropertyMap object with value type 'bool', for Graph 0x7f78628eade0, at 0x7f78613c54f0>, vertices filtered by <VertexPropertyMap object with value type 'bool', for Graph 0x7f78628eade0, at 0x7f78628eba70>, at 0x7f78628eade0>,
 <VertexPropertyMap object with value type 'vector<double>', for Graph 0x7f78628eade0, at 0x7f78613b00b0>)

In [35]:
plt.close("all")           # Close all open figures
# plt.switch_backend("agg")   # Revert to the default backend
plt.switch_backend("module://matplotlib_inline.backend_inline")  # Set to Jupyter inline backend

In [36]:
rand_code = state_title_filename.split('_')[-1]

title = topic_labels[topic]

import matplotlib

In [37]:

# edge width
ew = g.ep.eprob.copy()
ew.a = abs(ew.a - .5)

# get a color map
clrs = [(1, 0, 0, 1.0),
        (0, 0, 0, 1.0)]
red_cm = matplotlib.colors.LinearSegmentedColormap.from_list("Set3", clrs)


In [38]:
countries = []
for v in g.vertices():
    countries.append( g.vp.country[v] )

In [39]:
country_df = pd.DataFrame({
    'ISO_A3': countries,
    'block': country_colors
})

In [40]:
import geopandas as gpd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Load world map shapefile using geopandas
# world = gpd.read_file('shape-files/ne_110m_admin_0_countries.shp')

world = gpd.read_file('shape-files/ne_110m_admin_0_countries.shp')

# Handle potential discrepancies in country codes
world.loc[world['ADMIN'] == 'France', 'ISO_A3'] = 'FRA'
world.loc[world['ADMIN'] == 'Norway', 'ISO_A3'] = 'NOR'

# Reproject the world map to a different CRS (for visualization purposes)
world = world.to_crs(epsg=3395)

# Merge the world GeoDataFrame with the country_df based on the 'ISO_A3' country codes
world = world.merge(country_df, left_on='ISO_A3', right_on='ISO_A3', how='left')


In [41]:
# Initialize the `pos` property in the graph for storing (x, y) coordinates
pos = g.new_vertex_property("vector<double>")

# Iterate over vertices and assign centroid positions based on matching ISO_A3 codes
for v in g.vertices():
    country_code = g.vp.country[v]  # Get the ISO_A3 code from the vertex property

    # Find the centroid for the country from the world DataFrame
    centroid = world.loc[world['ISO_A3'] == country_code, 'geometry'].centroid

    # Assign the centroid coordinates to the `pos` property if found
    if not centroid.empty:
        pos[v] = (centroid.values[0].x, centroid.values[0].y)
        if g.vp.country[v] == 'USA':
            pos[v] = (-11438974.432, 4698615)
            print(pos[v])
    else:
        pos[v] = (0, 0)  # Assign a default position if no match is found

array([-11438974.432,   4698615.   ])


In [42]:
world['block'] = world['block'].astype('category')

# Get the number of unique categories in the 'continent' column
num_categories = len(world['block'].cat.categories)

In [43]:
title = 'ALL'

In [44]:
rand_code = 0000

In [45]:
# Create a discrete colormap from the tab20 colormap

from matplotlib.colors import ListedColormap
import matplotlib 
from mpl_toolkits.axes_grid1 import make_axes_locatable

plt.close("all")           # Close all open figures
plt.switch_backend("cairo")   # to enable vector drawing

# plt.switch_backend("module://matplotlib_inline.backend_inline")  # Set to Jupyter inline backend
fig, ax = plt.subplots()


tab20 = matplotlib.cm.tab20
custom_colormap = ListedColormap(tab20.colors[:num_categories])  # Use only the number of categories needed

world_plot = world.plot(column='block', legend=False, ax=ax, cmap=custom_colormap, edgecolor='k', alpha = 1, linewidth=0.3, zorder = -1)


plt.title(topic)

ax.set_xticks([])
ax.set_yticks([])

# gt.graph_draw(g, pos = pos, vertex_shape="pie", vertex_pie_fractions=pv, vertex_text = vprop_label,
#               edge_pen_width=gt.prop_to_size(g.ep.eprob, 0.001, 0.01, power=1), mplfig = ax)

# gt.graph_draw(filtered_graph, pos = pos, vertex_shape="pie", vertex_pie_fractions=pv,
#               vcmap=matplotlib.cm.Set1, edge_color=(.1,.1,.1,.1), vertex_pen_width=10, vertex_pen_color="black",
#               edge_pen_width=1, mplfig = ax)


# gt.graph_draw(
#     g,
#     pos=pos,
#     vertex_shape="circle",                # Use solid circles as the background
#     vertex_fill_color="black",            # Set to black for the outline
#     vertex_size=400000,                       # Slightly larger size to act as the border
#     edge_color=(.1, .1, .1, .1),          # Keep edges as before
#     edge_pen_width=1,
#     mplfig=ax
# )
# set1_colors = [matplotlib.cm.Set1(i) for i in range(9)]

# # Now draw the actual pie vertices on top of the black "background"
# gt.graph_draw(
#     g,
#     pos=pos,
#     vertex_shape="pie",
#     vertex_pie_fractions=pv,
#     # vcmap=matplotlib.cm.Set1,
#     vcmap=matplotlib.colors.ListedColormap(set1_colors),
#     vertex_size=300000,                       # Smaller size for the actual pie chart vertices
#     edge_color=(.1, .1, .1, .1),
#     edge_pen_width=1,
#     mplfig=ax
# )
fig.savefig( './temp' + topic + '-' + str(rand_code) + ".pdf")
fig.savefig( './temp' + topic + '-' + str(rand_code) + ".svg")
fig.savefig( './temp' + topic + '-' + str(rand_code) + ".png", dpi = 500)
plt.show()

/tmp/ipykernel_13517/2186500873.py:61: UserWarning: FigureCanvasCairo is non-interactive, and thus cannot be shown
  plt.show()
